## Elie NOUHRA - Projet matière "SVM et ANN" - Master 2 ECAP IAE NANTES
Ce Jupyter Notebook contient la partie code (Python 3.9.13) et la partie Markdown. 

### SUJET TRAITÉ

Ce dataset [Kaggle](https://www.kaggle.com/datasets/hopesb/student-depression-dataset/data) traite de la **dépression chez les étudiants en Inde** et vise à analyser, comprendre et prédire les niveaux de dépression à partir de diverses caractéristiques telles que les informations démographiques (âge, sexe), les facteurs scolaires (moyenne générale, pression ressentie), les habitudes de vie (durée de sommeil, habitudes alimentaires) et les antécédents de santé mentale (pensées suicidaires). Il comprend **27 901 observations** et **18 colonnes**, avec une **variable cible binaire**, `Depression_Status` (1 = dépression, 0 = pas de dépression). Bien que celui ayant publié le dataset n'ait pas précisé l'année et la source des données, il est probable qu'elles soient **récentes** et tirées d'**une enquête**, car la base de données a été mise à jour il y a quelques mois et semble provenir directement d'un questionnaire administré aux étudiants. Ce dataset est particulièrement pertinent pour la **recherche en santé mentale**, en offrant une **vue multidimensionnelle des facteurs contribuant à la dépression**, et pour le secteur éducatif, qui peut en tirer des enseignements pour **adapter ses mesures et soutenir le bien-être des étudiants**. 

### CHARGEMENT ET APERÇU DES DONNÉES

In [ ]:
# Loading the data
import pandas
data=pandas.read_csv('Student_Depression_Dataset.csv')

# Preview of the data
data.info()
print(data.head())
print(data.tail())

In [ ]:
# Reorder columns to place 'Depression' in the second position
cols = list(data.columns)
cols.insert(1, cols.pop(cols.index('Depression')))
data = data[cols]
data.head()

### VALEURS MANQUANTES

In [ ]:
# Check for missing values
import seaborn as sns

missing_values = data.isnull().sum()
print("Missing values in each column :\n", missing_values)

In [ ]:
# Remove the 3 observations with NA values
data_cleaned= data.dropna()
data_cleaned.info()

### ANALYSE EXPLORATOIRE 

##### Visualisation de la répartition des valeurs de la variable cible dépression (et modifications si nécessaires)

In [ ]:
# Distribution of depression
import matplotlib.pyplot as plt
sns.countplot(x='Depression', data=data_cleaned)

plt.title('Distribution of Depression')
plt.show()

depression_counts = data_cleaned['Depression'].value_counts()
print(depression_counts)

# Calculate the proportion of depression and non-depression
depression_proportion = data_cleaned['Depression'].value_counts(normalize=True)
print("\n", depression_proportion)

**Plus de la moitié (58.5%) des étudiants dans le dataset souffrent de dépression**.
Je n'effectuerai pas de rééquilibrage car l'écart de représentation entre les classes est modéré, et la taille importante de l'échantillon (27 898 observations) assure une distribution suffisante des deux classes. De plus, je préfère étudier la variable cible sans modification pour éviter toute perte d'information ou introduction d'un biais artificiel.

##### Visualisation de la répartion des valeurs prises par les variables explicatives catégorielles (et modifications si nécessaires)

In [ ]:
# Distribution of values for object-type variables
object_columns = data_cleaned.select_dtypes(include=['object']).columns

for column in object_columns:
    print(f"Répartition des valeurs pour la colonne '{column}':")
    print(data_cleaned[column].value_counts())
    print("\n")

Les colonnes `Gender`, `Have you ever had suicidal thoughts ?` et `Family History of Mental Illness` contiennent des valeurs binaires. Il serait plus approprié de les convertir en variables de type numérique (par exemple, 1 pour "Female" et 0 pour "Male").

In [ ]:
# Convert "Gender" to binary and rename it to "Female"
data_cleaned['Female'] = data_cleaned['Gender'].apply(lambda x: 1 if x == 'Female' else 0)
# Drop the original "Gender" column
data_cleaned = data_cleaned.drop(columns=['Gender'])

# Reorder columns to place 'Female' in the third position
cols = list(data_cleaned.columns)
cols.insert(2, cols.pop(cols.index('Female')))
data_cleaned = data_cleaned[cols]

# Convert "Have you ever had suicidal thoughts ?" to binary and rename it to "Suicidal Thoughts"
data_cleaned['Suicidal Thoughts'] = data_cleaned['Have you ever had suicidal thoughts ?'].apply(lambda x: 1 if x == 'Yes' else 0)
# Drop the original "Have you ever had suicidal thoughts ?" column
data_cleaned = data_cleaned.drop(columns=['Have you ever had suicidal thoughts ?'])

# Convert "Family History of Mental Illness" to binary
data_cleaned['Family History of Mental Illness'] = data_cleaned['Family History of Mental Illness'].apply(lambda x: 1 if x == 'Yes' else 0)

# Verify the changes
data_cleaned.head()

Passer d'un type `object` à `category` permet de réduire l'utilisation mémoire et d'optimiser les performances lors des opérations de filtrage, de regroupement ou d'agrégation, surtout lorsque la colonne contient un nombre limité de valeurs distinctes.

In [ ]:
# Convert relevant columns to 'category' dtype
categorical_columns = ['City', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree']
data_cleaned[categorical_columns] = data_cleaned[categorical_columns].astype('category')

# Verify the changes
print(data_cleaned.info())

In [ ]:
# Distribution of city
sns.countplot(x='City', data=data_cleaned)
plt.title('Distribution of City')
plt.xticks(rotation=90)
plt.show()

city_counts = data_cleaned['City'].value_counts()
print(city_counts)

In [ ]:
# Group cities with 2 or fewer observations

# Count the occurrences of each city
city_counts = data_cleaned['City'].value_counts()
# Identify cities with 2 or fewer observations
cities_to_replace = city_counts[city_counts <= 2].index
# Replace these cities with 'Others'
data_cleaned['City'] = data_cleaned['City'].replace(cities_to_replace, 'Others')
# Verify the changes
print(data_cleaned['City'].value_counts())

In [ ]:
# Distribution of sleep duration
sns.countplot(x='Sleep Duration', data=data_cleaned)
plt.title('Distribution of Sleep Duration')
plt.show()

unique_sleep_duration = data_cleaned['Sleep Duration'].unique()
print("Valeurs uniques prises par 'Sleep Duration':\n", unique_sleep_duration)
unique_sleep_duration = [duration.replace('Less than 5 hours', '< 5 hours').replace('More than 8 hours', '> 8 hours') for duration in unique_sleep_duration]
print("Valeurs uniques prises par 'Sleep Duration':\n", unique_sleep_duration)

sleep_duration = data_cleaned['Sleep Duration'].value_counts()
print(sleep_duration)

Je vais exlure la `catégorie Others` de la variable `Sleep Duration` car elle semble correspondre à des réponses manquantes ou à des individus qui ne souhaitent pas répondre, ce qui ne représente pas une information pertinente pour l'analyse. De plus, le faible nombre d'observations (18) pour cette catégorie par rapport à l'ensemble des données (27 898 individus) suggère qu'elle ne contribuerait pas de manière significative à la modélisation et pourrait introduire du bruit dans les résultats.

In [ ]:
data_cleaned = data_cleaned[data_cleaned['Sleep Duration'] != 'Others']
data_cleaned['Sleep Duration'] = data_cleaned['Sleep Duration'].cat.remove_categories('Others')
data_cleaned.info()

In [ ]:
# Distribution of dietary habits
sns.countplot(x='Dietary Habits', data=data_cleaned)
plt.title('Distribution of Dietary Habits')
plt.xticks(rotation=90)
plt.show()

dietary_habits_counts = data_cleaned['Dietary Habits'].value_counts()
print(dietary_habits_counts)

In [ ]:
# Exclure catégorie "Others" (Cf. justification de l'exclusion de la catégorie "Others" de la variable "Sleep Duration")
data_cleaned = data_cleaned[data_cleaned['Dietary Habits'] != 'Others'] 
data_cleaned['Dietary Habits'] = data_cleaned['Dietary Habits'].cat.remove_categories('Others')
data_cleaned.info()

In [ ]:
# Distribution of profession
sns.countplot(x='Profession', data=data_cleaned)
plt.title('Distribution of Profession')
plt.xticks(rotation=90)
plt.show()

profession_counts = data_cleaned['Profession'].value_counts()
print(profession_counts)

Je vais supprimer la variable `Profession` car mon sujet porte spécifiquement sur la dépression chez les étudiants (il y a probablement eu une erreur lors de la collecte ou de l'enregistrement des données).

In [16]:
data_cleaned = data_cleaned.drop(columns=['Profession'])

In [ ]:
# Distribution of degree
sns.countplot(x='Degree', data=data_cleaned)
plt.title('Distribution of Degree')
plt.xticks(rotation=90)
plt.show()

degree_counts = data_cleaned['Degree'].value_counts()
print(degree_counts)

In [ ]:
# Répartition des valeurs des variables de type category après modifications
object_columns = data_cleaned.select_dtypes(include=['category']).columns

for column in object_columns:
    print(f"Répartition des valeurs pour la colonne '{column}':")
    print(data_cleaned[column].value_counts())
    print("\n")

##### Visualisation de la répartion des valeurs prises par les variables explicatives quantitatives (et modifications si nécessaires)

In [ ]:
# Sélectionner les X quantitatifs
quantitative_columns = data_cleaned.select_dtypes(include=['float64', 'int32','int64']).columns
quantitative_columns = quantitative_columns.difference(['Depression','id'])

for column in quantitative_columns:
    print(f"Répartition des valeurs pour la colonne '{column}':")
    print(data_cleaned[column].describe())
    print("\n")

Plusieurs variables lues comme float correspondent en fait à des variables binaires ou à des échelles de valeurs discrètes. Par exemple `Financial Stress` varie de 0 (pas de stress ressenti) à 5 (stress ressenti très élevé).
La moyenne académique générale (`CPGA`) est une échelle de valeurs continues. 

In [20]:
# Convertir en integer les variables quantitatives discrètes (c'est-dire-dire toutes sauf CGPA)
data_cleaned[quantitative_columns.difference(['CGPA'])] = data_cleaned[quantitative_columns.difference(['CGPA'])].astype('int')

In [ ]:
# Suppression de la variable Job Satisfaction et Work Pressure non pertinentes pour l'analyse (Cf. justification de l'exclusion de la variable "Profession")
data_cleaned.info()
data_cleaned = data_cleaned.drop(columns=['Job Satisfaction', 'Work Pressure']) 

In [ ]:
# Rename the column 'Work/Study Hours' to 'Study Hours'
data_cleaned = data_cleaned.rename(columns={'Work/Study Hours': 'Study Hours'})

# Verify the changes
print(data_cleaned.info())

In [ ]:
## Aperçu du jeu de données après modifications (33 observations et trois variables écartées)
data_cleaned.info()
data_cleaned.head()

##### Détection et traitement des valeurs atypiques

In [ ]:
# Sélectionner les X quantitatifs
quantitative_columns = data_cleaned.select_dtypes(include=['float64', 'int32','int64']).columns
quantitative_columns = quantitative_columns.difference(['Depression','id'])
quantitative_columns = quantitative_columns.difference(['Family History of Mental Illness', 'Female', 'Suicidal Thoughts']) # Exclude binary variables from the quantitative columns

# Calculer et afficher le nombre de valeurs distinctes pour chaque colonne quantitative
distinct_values = data_cleaned[quantitative_columns].nunique()
print("Nombre de valeurs distinctes pour les colonnes quantitatives :")
print(distinct_values)

# Afficher les caractéristiques de base pour les  X quantitatifs
print(data_cleaned.drop(columns=['Depression', 'id', 'Family History of Mental Illness', 'Female', 'Suicidal Thoughts']).describe())

In [ ]:
# Visualisation des distributions des X quantitatifs à l'aide de boxplots
import matplotlib.pyplot as plt

# Définir une palette de couleurs
palette = sns.color_palette("husl", len(quantitative_columns))  # Palette Husl pour des couleurs variées

# Créer un boxplot esthétique pour chaque colonne quantitative
for i, column in enumerate(quantitative_columns):
    plt.figure(figsize=(8, 6))  # Taille de la figure
    sns.boxplot(
        data=data_cleaned, 
        x=column, 
        color=palette[i],  # Couleur spécifique pour chaque variable
        width=0.6,         # Largeur du boxplot
        saturation=0.8     # Transparence pour un rendu plus doux
    )
    plt.title(f'Boxplot de la variable {column}', fontsize=14, fontweight='bold', color=palette[i])
    plt.xlabel(column, fontsize=12)
    plt.ylabel('Valeurs', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)  # Grille discrète
    plt.tight_layout()  # Ajustement pour éviter les chevauchements
    plt.show()    

**Valeurs atypiques** détectées pour l'`âge` (étudiants âgés) et la `moyenne académique globale` (moyenne académique nulle) 

In [ ]:
# Summary statistics for Age and CGPA
import pandas as pd

# Summary statistics for Age
age_summary = data_cleaned['Age'].describe()
quantiles = data_cleaned['Age'].quantile([0.80, 0.90, 0.95, 0.99])
age_summary = pd.concat([age_summary, quantiles])
# Replace quantile values with percentages
age_summary.index = age_summary.index.map(lambda x: f"{int(x * 100)}%" if isinstance(x, float) else x)
print("Summary statistics for Age with percentages:\n", age_summary)

# Summary statistics for CGPA
cgpa_summary = data_cleaned['CGPA'].describe()
quantiles = data_cleaned['CGPA'].quantile([0.05,0.10,0.20,0.80, 0.90, 0.95, 0.99])
cgpa_summary = pd.concat([cgpa_summary, quantiles])
# Replace quantile values with percentages
cgpa_summary.index = cgpa_summary.index.map(lambda x: f"{int(x * 100)}%" if isinstance(x, float) else x)
print("\nSummary statistics for CGPA with percentages:\n", cgpa_summary)

In [ ]:
# Fonction détection outliers
def detect_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

# Detect outliers for Age
age_outliers = detect_outliers(data_cleaned, 'Age')
print("Outliers for Age:\n", age_outliers[['Age']])

# Detect outliers for CGPA
cgpa_outliers = detect_outliers(data_cleaned, 'CGPA')
print("Outliers for CGPA:\n", cgpa_outliers[['CGPA']])

# Exclude outliers for Age
data_cleaned = data_cleaned[~data_cleaned.index.isin(age_outliers.index)]

# Exclude outliers for CGPA
data_cleaned = data_cleaned[~data_cleaned.index.isin(cgpa_outliers.index)]

# Verify the changes
print("Data after excluding outliers:")
print(data_cleaned.info())
print(data_cleaned.describe())

# Number of individuals removed for Age
removed_for_age = len(age_outliers)
print(f"Number of individuals removed for Age: {removed_for_age}")

# Number of individuals removed for CGPA
removed_for_cgpa = len(cgpa_outliers)
print(f"Number of individuals removed for CGPA: {removed_for_cgpa}")

# Total number of individuals removed
total_removed = len(age_outliers.index.union(cgpa_outliers.index))
print(f"Total number of individuals removed: {total_removed}")

Il est logique d'avoir **enlevé ces 21 étudiants aux caractéristiques incontestablement singulières** (personnes de 44 ans ou plus et individus ayant une moyenne générale nulle, peut-être car ils ne se sont jamais rendus aux cours et examens).

In [ ]:
# Boxplot of Age after modifications
plt.figure(figsize=(8, 6))
sns.boxplot(data=data_cleaned, x='Age', color='skyblue', width=0.6, saturation=0.8)
plt.title("Distribution de l'âge des étudiants sans les outliers", fontsize=14, fontweight='bold', color='skyblue')
plt.ylabel('Valeurs', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Boxplot of CGPA after modifications
plt.figure(figsize=(8, 6))
sns.boxplot(data=data_cleaned, x='CGPA', color='lightgreen', width=0.6, saturation=0.8)
plt.title("Distribution de la moyenne académique globale sans les outliers", fontsize=14, fontweight='bold', color='lightgreen')
plt.xlabel('CGPA', fontsize=12)
plt.ylabel('Valeurs', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

##### Détection et traitement des liens significatifs entre les variables explicatives

Entre variables quantitatives continues

In [ ]:
from scipy.stats import pearsonr, spearmanr

# Sélectionner les X quantitatifs
quantitative_columns = data_cleaned.select_dtypes(include=['float64', 'int32','int64']).columns
quantitative_columns = quantitative_columns.difference(['Depression','id'])

# Scatterplot between Age and CGPA
sns.scatterplot(data=data_cleaned, x='Age', y='CGPA')
plt.title('Scatterplot between Age and CGPA')
plt.xlabel('Age')
plt.ylabel('CGPA')
plt.show()

# Calculer la corrélation de Pearson (vérifier si relation linéaire)
pearson_corr, pearson_p_value = pearsonr(data_cleaned['Age'], data_cleaned['CGPA'])
print(f"Corrélation de Pearson entre l'âge et le CGPA: {pearson_corr:.4f} (p-value: {pearson_p_value:.4e})")

# Calculer la corrélation de Spearman (vérifier si relation monotone, qu'elle soit linéaire ou non)
spearman_corr, spearman_p_value = spearmanr(data_cleaned['Age'], data_cleaned['CGPA'])
print(f"Corrélation de Spearman entre l'âge et le CGPA: {spearman_corr:.4f} (p-value: {spearman_p_value:.4e})")

print("\nConclusion : Pas de lien significatif")

Entre variables quantitatives

In [ ]:
from scipy.stats import spearmanr
import pandas as pd

# Calculer la matrice de corrélation de Spearman
correlation_matrix = data_cleaned[quantitative_columns].corr(method='spearman')

# Initialiser un DataFrame pour stocker les p-values
p_values = pd.DataFrame(0, index=quantitative_columns, columns=quantitative_columns, dtype=float)

# Calculer les p-values pour chaque paire de variables
for col1 in quantitative_columns:
    for col2 in quantitative_columns:
        if col1 != col2:
            _, p_value = spearmanr(data_cleaned[col1], data_cleaned[col2])
            p_values.loc[col1, col2] = p_value

# Filtrer les corrélations significatives (p-value < 0.05) et les coefficients > 0.25
significant_correlations = correlation_matrix[
    (p_values < 0.05) & (p_values != 0) & (correlation_matrix.abs() > 0.25)
]

# Afficher les corrélations significatives
print("Significant Spearman correlations (p-value < 0.05 and |correlation| > 0.25):")
print(significant_correlations)

print("\nConclusion : Pas de liens significatifs")

Entre variables quantitatives et catégorielles

In [ ]:
data_cleaned.info()

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Rename columns to remove spaces
data_cleaned.columns = data_cleaned.columns.str.replace(' ', '_')

# Update the lists of quantitative and categorical columns
quantitative_columns = [col.replace(' ', '_') for col in quantitative_columns]
categorical_columns = ['City', "Sleep_Duration", "Dietary_Habits", 'Degree']

# Dictionnaire pour stocker les résultats de l'ANOVA
anova_results = {}

# Seuil de signification
alpha = 0.05

# Effectuer l'ANOVA pour chaque paire de variable quantitative et catégorielle
for quant_col in quantitative_columns:
    for cat_col in categorical_columns:
        formula = f'{quant_col} ~ C({cat_col})'
        model = ols(formula, data=data_cleaned).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        
        # Vérifier si la p-value est inférieure au seuil
        p_value = anova_table['PR(>F)'].iloc[0]  # p-value pour la première ligne (celle de la variable catégorielle)
        if p_value < alpha:
            anova_results[(quant_col, cat_col)] = anova_table

# Afficher les résultats de l'ANOVA pour les liens significatifs
for key, result in anova_results.items():
    print(f'ANOVA results for {key[0]} ~ {key[1]} (p-value < {alpha}):')
    print(result)
    print('\n')

In [ ]:
print("Les résultats de l'ANOVA montrent qu'un grand nombre de variables, telles que la pression académique et le CGPA, sont significativement liées à des facteurs comme le le diplôme, la durée de sommeil, etc.")
print("\nPrenons l'exemple des résultats de l'ANOVA pour 'Academic_Pressure ~ City', où nous obtenons une p-value largement inférieure à 0.01 (donc bien en dessous du seuil de 5%).")
print("Cela indique qu'il existe une différence statistiquement significative dans la pression académique entre les différentes villes. En d'autres termes, le lieu (la ville) a un impact sur la pression académique.")
print("\nCes relations peuvent entraîner des problèmes de multicolinéarité dans les estimations économétriques, car certaines variables pourraient être fortement corrélées entre elles.")
print("Pour vérifier la multicolinéarité, nous utiliserons le VIF (Variance Inflation Factor) à l'étape de modélisation.")

Entre variables catégorielles

In [ ]:
from scipy.stats import chi2_contingency

# Dictionnaire pour stocker les résultats du test du khi-deux
chi2_results = {}

# Définir un seuil alpha pour la significativité
alpha = 0.05

# Effectuer le test du khi-deux pour chaque paire de variables catégorielles
for i, col1 in enumerate(categorical_columns):
    for col2 in categorical_columns[i+1:]:  # Tester uniquement une fois chaque paire
        contingency_table = pd.crosstab(data_cleaned[col1], data_cleaned[col2])
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        
        # Si la p-value est inférieure au seuil alpha, enregistrer les résultats
        if p < alpha:
            chi2_results[(col1, col2)] = {'chi2': chi2, 'p-value': p, 'dof': dof}

# Afficher les résultats du test du khi-deux pour les liens significatifs
if chi2_results:
    for key, result in chi2_results.items():
        print(f'Test du khi-deux pour {key[0]} et {key[1]} (p-value < {alpha}):')
        print(f"Chi2: {result['chi2']}, p-value: {result['p-value']}, dof: {result['dof']}")
        print('\n')
else:
    print("Aucune relation significative trouvée entre les variables catégorielles.")

print("\n Conclusion : Relations significatives relevées, cela renforce l'importance de tester la multicolinéarité.")

Nous pouvons désormais analyser plus en profondeur les valeurs prises par les variables.

### ANALYSE DESCRIPTIVE 

In [ ]:
data_cleaned.info()
data_cleaned.head()

##### De la dépression (variable à expliquer)

In [ ]:
# Calculate the proportion of depression and non-depression
depression_proportion = data_cleaned['Depression'].value_counts(normalize=True)
print("Proportion of Depression and Non-Depression:\n", depression_proportion)

print("\nConclusion : Presque aucun changement de répartition par rapport au dataframe originel")

##### Analyse descriptive des autres informations sur les étudiants (variables à expliquer)

X quantitatifs

In [ ]:
# Sélectionner les X quantitatifs
quantitative_columns = data_cleaned.select_dtypes(include=['float64', 'int32','int64']).columns
quantitative_columns = quantitative_columns.difference(['Depression','id'])

# Calculer et afficher le nombre de valeurs distinctes pour chaque colonne quantitative
distinct_values = data_cleaned[quantitative_columns].nunique()
print("\nNombre de valeurs distinctes pour les colonnes quantitatives :")
print(distinct_values)

# Afficher les caractéristiques de base pour les  X quantitatifs
print(data_cleaned.drop(columns=['Depression', 'id']).describe())

In [ ]:
# Calculer les 10 valeurs les plus fréquentes et leur fréquence relative
for column in quantitative_columns:
    # Obtenir les 10 premières valeurs les plus fréquentes
    top_values = data_cleaned[column].value_counts().head(10)
    
    # Afficher chaque valeur et sa fréquence relative
    print(f"\nLes valeurs les plus fréquentes pour {column}:")
    for value, count in top_values.items():
        relative_frequency = count / len(data_cleaned)
        print(f" - {value}: {relative_frequency:.2%}")

L’échantillon est composé de **27 847 observations**, avec une répartition de **44 % de femmes et 56 % d'hommes**. L’`âge` varie de **18 à 43 ans** et la moyenne est de **26 ans**. Le **`CGPA` moyen et médian est d'approximativement 7.7** (min : 5, max : 10), avec des valeurs élevées relativement fréquentes. Les étudiants consacrent en moyenne **7h10** par jour aux études, et plus de **50 % étudient entre 6 et 10 heures**. La **`satisfaction aux études` est assez hétérogène**, avec les niveaux 1, 2, 3 et 4 représentant environ 20 % chacun.  

La **`pression académique` moyenne est de 3.1 sur 5, mais 41 % des étudiants s'identifient aux deux intensités les plus fortes**. Le **`stress financier ressenti` est similaire** (moyenne de 3.1 sur 5 avec 45 % d'étudiants qui déclarent un stress au plus haut niveau ou juste en dessous). Environ **48 % ont des `antécédents familiaux de troubles mentaux`**, et **63 % déclarent avoir eu des `pensées suicidaires`**, une statistique préoccupante.  

Ces éléments mettent en lumière le fait que le **panel d'individus interrogés semble assez anxieux et fragile**. On peut **suspecter que ces caractéristiques se reflètent dans la dépression**.

Graphiques

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Create boxplots for 'Age' and 'CGPA'
for column in ['Age', 'CGPA']:
    plt.figure(figsize=(8, 6))
    sns.boxplot(
        data=data_cleaned, 
        x=column, 
        color=palette[quantitative_columns.get_loc(column)],  # Use the same color palette
        width=0.6, 
        saturation=0.8
    )
    plt.title(f'Boxplot de la variable {column}', fontsize=14, fontweight='bold', color=palette[quantitative_columns.get_loc(column)])
    plt.xlabel(column, fontsize=12)
    plt.ylabel('Valeurs', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


## Créer des barplots pour chaque colonne sélectionnée

# Liste des colonnes à exclure
exclude_columns = ['Age', 'CGPA', 'Family_History_of_Mental_Illness', 'Female', 'Suicidal_Thoughts' ]
# Sélectionner les colonnes à inclure
quantitative_columns_to_plot = data_cleaned.select_dtypes(include=['int32', 'float64']).columns.difference(exclude_columns)

for column in quantitative_columns_to_plot:
    plt.figure(figsize=(10, 6))
    # Calculer la fréquence relative
    relative_freq = data_cleaned[column].value_counts(normalize=True)
    sns.barplot(x=relative_freq.index, y=relative_freq.values, palette='husl')
    plt.title(f'Fréquence relative de la variable {column}', fontsize=14, fontweight='bold')
    plt.xlabel(column, fontsize=12)
    plt.ylabel('Fréquence relative', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

Nos précédents commentaires se reflètent dans ces graphiques.

X qualitatifs

In [ ]:
# Afficher les caractéristiques de base pour les colonnes catégorielles
categorical_columns = ['City', 'Sleep_Duration', 'Dietary_Habits', 'Degree']
data_cleaned[categorical_columns] = data_cleaned[categorical_columns].astype('category')
print(data_cleaned.info())
print(data_cleaned.describe(include=['category']))

In [ ]:
# Calculer les 10 valeurs les plus fréquentes et leur fréquence relative pour chaque variable catégorielle
for column in categorical_columns:
    # Obtenir les 10 premières valeurs les plus fréquentes
    top_values = data_cleaned[column].value_counts().head(10)
    
    # Afficher chaque valeur et sa fréquence relative
    print(f"\nLes valeurs les plus fréquentes pour {column}:")
    for value, count in top_values.items():
        relative_frequency = count / len(data_cleaned)
        print(f" - {value}: {relative_frequency:.2%}")

L'échantillon couvre **31 `villes` différentes**, avec Kalyan en tête (5,6 % des réponses), suivie de Srinagar (4,9 %) et Hyderabad (4,8 %), suggérant une **représentation géographique relativement large**, bien que certaines villes apparaissent un peu plus fréquemment. **La `durée du sommeil` est fortement hétérogène**. En effet, bien que **30 % des étudiants dorment moins de 5 heures par nuit**, les autres groupes sont également bien représentés (entre 22 et 27 %). Pour les **`habitudes alimentaires`, une grande majorité des étudiants ont des habitudes considérées comme non optimales**, avec environ 37 % ayant des habitudes "non saines", suivis de ceux ayant des habitudes "modérées" (35,6 %). Seuls 27,4 % des étudiants adoptent un régime alimentaire "sain". Ces chiffres peuvent être liés à l'anxiété et au bien-être plus généralement. Enfin, concernant les **28 `diplômes`**, **plus d'un cinquième des répondants sont en Classe 12** (**21,8 %**), suivis par ceux en B.Ed (6,7 %) et B.Com (5,4 %). Les trois cursus évoqués correspondent respectivement à la dernière année du secondaire, un diplôme de premier cycle formant les enseignants et une licence en commerce et gestion. L'ensemble de ces éléments permet de mieux comprendre les caractéristiques de l'échantillon et soulève à nouveau des **tendances préoccupantes pour une part non négligeable des étudiants**, notamment en ce qui concerne la durée du sommeil et les habitudes alimentaires.

Ces observations permettent de mieux comprendre la situation mentale et le cadre de vie des étudiants de l'échantillon. 
Nous pouvons à présent passer à l'estimation de modèles.

### MODÉLISATION

In [ ]:
# Afficher les informations du DataFrame
print(data_cleaned.info())

# Statistiques descriptives pour les variables quantitatives
print("Statistiques descriptives pour les variables quantitatives :")
print(data_cleaned[quantitative_columns].describe())

# Fréquences pour les variables catégorielles
print("\nFréquences pour les variables catégorielles :")
for column in categorical_columns:
    print(f"\nValeurs pour la colonne '{column}':")
    print(data_cleaned[column].value_counts())

Pour la modélisation je vais m'appuyer sur **deux méthodes**, la `régression logistique avec le package statsmodels` et le `Gradient Boosting avec la bibliothèque XGBoost`. J'ai choisi la première car elle est parfaitement adaptée à un problème de classification binaire et est aisément interprétable. En complément, XGBoost est un modèle plus complexe et puissant qui peut capturer des relations non linéaires et gérer des interactions complexes entre les variables, offrant ainsi une meilleure performance dans des contextes de grande dimension et de données complexes. **Ensemble, ces deux modèles permettent de comparer la précision et l'interprétabilité d'approches simples et avancées**. 
Après avoir préparé les données, nous estimerons les modèles, les comparerons à l'aide de divers indicateurs de performance puis nous en interpréterons les résultats en essayant d'identifier dans quelle mesure certains facteurs sont liés à la dépression des étudiants interrogés.

##### Préparation des données

In [ ]:
# Encodage one-hot des variables catégorielles
data_encoded = pd.get_dummies(data_cleaned, columns=['Dietary_Habits', 'Sleep_Duration','Degree', 'City'], drop_first=True)

# Verify the changes
print(data_encoded.head())

Bien que la `régression logistique` puisse bénéficier de la standardisation, dans mon cas, **les variables n'ont pas de différences d'échelle extrêmes**. En outre garder les valeurs telles quelles permet de **faciliter l'interprétation**. 

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

# Define the features (X) and the target (y)
X = data_encoded.drop(columns=['Depression', 'id'])
y = data_encoded['Depression']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the shapes of the resulting datasets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

# Vérification des informations des DataFrames 
print(data_encoded.info())
print(X_train.info())

##### Régression logistique binaire (bibliothèque statsmodels)

In [ ]:
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

# Convert boolean columns to integers
X_train = X_train.astype(int)

# Ajouter une constante pour l'interception
X_train_const = sm.add_constant(X_train)

# Créer le modèle de régression logistique
logit_model = sm.Logit(y_train, X_train_const)

# Ajuster le modèle
result = logit_model.fit()

# Prédire les valeurs pour l'ensemble de test
X_test_const = sm.add_constant(X_test)

# Ensure there are no invalid values in X_test_const
X_test_const = X_test_const.astype(float)

Vérification de la multicolinéarité à l'aide du `VIF` (`Variance Inflation Factor`)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = X_train_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]

# Trier le DataFrame par VIF de manière décroissante
vif_data_sorted = vif_data.sort_values(by="VIF", ascending=False)

# Changer la configuration d'affichage pour afficher toutes les lignes
pd.set_option('display.max_rows', None)  # Afficher toutes les lignes
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes

# Afficher le DataFrame trié
print(vif_data_sorted)

# Réinitialiser les options pour revenir à la configuration par défaut
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

Les villes posent clairement problème. Nous allons les regrouper selon leur taille (population) en nous appuyant sur la source officielle pour les données démographiques en Inde, à savoir [Census of India](https://censusindia.gov.in/census.website/). Nous avons choisi de les séparer en trois catégories : petites villes (moins de 1 million d'habitants), villes moyennes (entre 1 et 5 millions d'habitants) et grandes villes (plus de 5 millions d'habitants).

In [ ]:
# Create a dictionary to map cities to their sizes
city_size_mapping = {
    "Faridabad": "Small",
    "Ghaziabad": "Small",
    "Rajkot": "Small",
    "Indore": "Small",
    "Kanpur": "Small",
    "Meerut": "Small",
    "Varanasi": "Small",
    "Nashik": "Small",
    "Nagpur": "Small",
    "Ahmedabad": "Medium",
    "Pune": "Medium",
    "Surat": "Medium",
    "Jaipur": "Medium",
    "Lucknow": "Medium",
    "Vadodara": "Medium",
    "Bhopal": "Medium",
    "Patna": "Medium",
    "Ludhiana": "Medium",
    "Srinagar": "Medium",
    "Visakhapatnam": "Medium",
    "Kalyan": "Medium",
    "Thane": "Medium",
    "Mumbai": "Large",
    "Delhi": "Large",
    "Bangalore": "Large",
    "Hyderabad": "Large",
    "Chennai": "Large",
    "Kolkata": "Large"
}

# Map cities to their sizes
data_cleaned['City_Size'] = data_cleaned['City'].map(city_size_mapping)

# Verify the changes
print(data_cleaned[['City', 'City_Size']].head())

# Encodage one-hot des variables catégorielles
data_encoded = pd.get_dummies(data_cleaned, columns=['Dietary_Habits', 'Sleep_Duration','Degree', 'City_Size'], drop_first=True)

# Remove the 'City' variable from data_encoded
data_encoded = data_encoded.drop(columns=['City'])

# Verify the changes
print(data_encoded.head())

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

# Define the features (X) and the target (y)
X = data_encoded.drop(columns=['Depression', 'id'])
y = data_encoded['Depression']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the shapes of the resulting datasets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

# Vérification des informations des DataFrames 
print(data_encoded.info())
print(X_train.info())

In [ ]:
# Réestimation du modèle de régression logistique
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Convert boolean columns to integers
X_train = X_train.astype(int)

# Ajouter une constante pour l'interception
X_train_const = sm.add_constant(X_train)

# Créer le modèle de régression logistique
logit_model = sm.Logit(y_train, X_train_const)

# Ajuster le modèle
result = logit_model.fit()

# Prédire les valeurs pour l'ensemble de test
X_test_const = sm.add_constant(X_test)

# Ensure there are no invalid values in X_test_const
X_test_const = X_test_const.astype(float)

y_pred = result.predict(X_test_const)
y_pred_class = (y_pred >= 0.5).astype(int)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = X_train_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]

# Trier le DataFrame par VIF de manière décroissante
vif_data_sorted = vif_data.sort_values(by="VIF", ascending=False)

# Changer la configuration d'affichage pour afficher toutes les lignes
pd.set_option('display.max_rows', None)  # Afficher toutes les lignes
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes

# Afficher le DataFrame trié
print(vif_data_sorted)

# Réinitialiser les options pour revenir à la configuration par défaut
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

Tous les `VIF` des variables sont à présent **inférieurs à 5, ce qui indique une faible multicolinéarité entre les facteurs**, garantissant des estimations plus stables et interprétables.

In [ ]:
# Indicateurs de performance du modèle 

y_pred = result.predict(X_test_const)
y_pred_class = (y_pred >= 0.5).astype(int)

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred_class)
print("Confusion Matrix:\n", conf_matrix)

# Précision
accuracy = accuracy_score(y_test, y_pred_class)
print("Accuracy:", accuracy)

# Précision (Precision)
precision = precision_score(y_test, y_pred_class)
print("Precision:", precision)

# Rappel (Recall)
recall = recall_score(y_test, y_pred_class)
print("Recall:", recall)

# Score F1
f1 = f1_score(y_test, y_pred_class)
print("F1 Score:", f1)

# AUC-ROC
roc_auc = roc_auc_score(y_test, y_pred)
print("AUC-ROC:", roc_auc)

print("\nNous comparerons ces résultats avec ceux du modèle XGBoost.")

##### Gradient Boosting (bibliothèque XGboost)

Estimer un modèle XGBoost avec validation croisée (CV) plutôt qu'avec les paramètres par défaut permet, en théorie, d'obtenir une évaluation plus robuste de sa performance et d'optimiser ses hyperparamètres. C'est ce que nous allons vérifier. 
NB : Il n'est pas nécessaire de vérifier la multicolinéarité pour XGBoost car il s'agit d'un modèle basé sur des arbres de décision, qui sélectionnent les variables de manière séquentielle et ne sont pas affectés par la colinéarité entre les prédicteurs.

In [ ]:
# Modèle par défaut
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Create the XGBoost model with default parameters
xgb_model = XGBClassifier(random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Predict the values for the test set
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)

print("XGBoost Model Performance:")
print("Accuracy:", accuracy_xgb)
print("Precision:", precision_xgb)
print("Recall:", recall_xgb)
print("F1 Score:", f1_xgb)
print("AUC-ROC:", roc_auc_xgb)

In [ ]:
# Avec CV 
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.stats import uniform, randint

# Définir les distributions des hyperparamètres à tester
param_dist = {
    'max_depth': randint(3, 10),  # Limiter la profondeur des arbres
    'learning_rate': uniform(0.01, 0.3),  # Distribution continue pour un learning rate flexible
    'n_estimators': randint(100, 500),  # Limiter le nombre d'estimations pour éviter le surajustement
    'subsample': uniform(0.7, 0.3),  # Subsampling entre 0.7 et 1
    'colsample_bytree': uniform(0.7, 0.3),  # Exploration continue pour le colsample_bytree
    'alpha': uniform(0, 1),  # Ajouter de la régularisation L1
    'lambda': uniform(0, 1)  # Ajouter de la régularisation L2
}

# Créer le modèle XGBoost 
xgb_model = XGBClassifier(random_state=42)  

# Appliquer RandomizedSearchCV avec validation croisée 
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=100, cv=4, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)

# Entraîner le modèle avec les meilleurs hyperparamètres trouvés par la recherche aléatoire (le code a mis 2 minutes à s'exécuter sur mon ordinateur)
random_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres trouvés
print("Best parameters found: ", random_search.best_params_)

# Prédire les valeurs pour l'ensemble de test avec le meilleur modèle
y_pred_xgb = random_search.best_estimator_.predict(X_test)
y_pred_proba_xgb = random_search.best_estimator_.predict_proba(X_test)[:, 1]

# Évaluer les performances du modèle
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)

# Afficher les résultats
print("XGBoost Model Performance with RandomizedSearchCV:")
print("Accuracy:", accuracy_xgb)
print("Precision:", precision_xgb)
print("Recall:", recall_xgb)
print("F1 Score:", f1_xgb)
print("AUC-ROC:", roc_auc_xgb)

**L'optimisation des hyperparamètres a permis d'améliorer légérement l'ensemble des performances** (par exemple la capacité discriminante avec l'AUC-ROC).

Rappel des indicateurs de performance de la régression logistique : 
Accuracy: 0.8468581687612208
Precision: 0.8492829967808019
Recall: 0.895679012345679
F1 Score: 0.871864203094487
AUC-ROC: 0.9227828114237271

Bien que XGBoost soit un modèle plus complexe et généralement plus puissant, dans le cas présent, les **deux méthodes sont au coude-à-coude en termes de performance**. Cela pourrait s’expliquer par le fait que nous traitons un problème de classification binaire, qui s’aligne parfaitement avec l’objectif premier d’une régression logistique. **Cette dernière est le meilleur choix ici en raison de son interprétabilité, largement supérieure à celle du gradient boosting**. Elle permet une analyse plus transparente des effets des facteurs sur la variable cible, un critère d’autant plus crucial dans le cadre de la dépression, que nous cherchons à prédire mais aussi à expliquer. Nous interpréterons tout de même le modèle XGBoost afin de respecter les consignes.


##### Interprétations des résultats 

##### Interprétation locale et globale du modèle XGBoost par défaut (conformément aux consignes)

**Interprétation globale** : `Feature importance` (mesure l'effet d'une feature sur la fonction de perte) et `SHapley Additive exPlanations (SHAP) Global` (mesure l'impact moyen d'une caractéristique sur les prédictions)

In [ ]:
# Feature importance
import matplotlib.pyplot as plt
import pandas as pd

# Récupérer le booster du meilleur modèle
booster = random_search.best_estimator_.get_booster()

# Obtenir les scores d'importance des caractéristiques
importance = booster.get_score(importance_type='gain')  # 'weight', 'gain' ou 'cover'

# Convertir en DataFrame
importance_df = pd.DataFrame(importance.items(), columns=['Feature', 'Importance'])

# Trier les caractéristiques par ordre décroissant d'importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Sélectionner les 15 principales caractéristiques
top_features = importance_df.head(15)

# Afficher le tableau
print(top_features)

# Visualiser avec un graphique amélioré
plt.figure(figsize=(12, 8))
plt.barh(top_features['Feature'], top_features['Importance'], color='teal')
plt.xlabel('Importance (Gain)')
plt.ylabel('Features')
plt.title('Top 15 Feature Importance using XGBoost')
plt.gca().invert_yaxis()  # Mettre la plus importante en haut
plt.xticks(rotation=45)  # Rotation pour améliorer la lisibilité
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

Les résultats montrent que **certaines caractéristiques ont un impact significatif sur la prédiction du modèle**, notamment la variable `Suicidal_Thoughts` qui apparaît comme la plus influente, avec une importance de 260,25. Cela indique clairement que **la pensée suicidaire joue un rôle central dans la prédiction de la dépression chez les étudiants**, comme on pouvait s'y attendre. Ensuite, la `Academic_Pressure` (98,42) et le `Financial_Stress` (57,06) se révèlent également cruciaux, bien que leur impact soit inférieur à celui des pensées suicidaires. Cela suggère que **la pression académique et les difficultés financières contribuent de manière significative à la détérioration de la santé mentale**. En revanche, des variables comme `Dietary_Habits_Unhealthy` (26,57) et `Age` (15,04), bien que pertinentes, jouent un rôle moins déterminant. D'autres facteurs tels que `Degree_M.Com` (5,02) et `City_Size_Medium` (4,55) ont une importance relativement faible, ce qui suggère qu'ils ont un effet secondaire ou moins direct sur les prédictions. En résumé, **les variables liées à la santé mentale, à la pression académique et au stress sont les plus déterminantes**, ce qui confirme leur rôle primordial dans le modèle.

In [ ]:
# SHAP Global
import shap
import matplotlib.pyplot as plt

# Créer un explainer SHAP pour XGBoost
explainer = shap.Explainer(random_search.best_estimator_)

# Calculer les valeurs SHAP pour l'ensemble des données d'entraînement
shap_values = explainer(X_train)

# Afficher un beeswarm plot
shap.initjs()  # Initialisation de JavaScript pour le rendu du plot
shap.plots.beeswarm(shap_values)
plt.show()

# Calculer la moyenne des valeurs SHAP pour chaque variable
mean_shap_values = shap_values.values.mean(axis=0)

# Créer un DataFrame pour stocker les résultats
shap_summary = pd.DataFrame({
    'Feature': X_train.columns,  # Nom de la caractéristique
    'Mean SHAP': mean_shap_values  # Moyenne des valeurs SHAP (avec signe)
})

# Trier par ordre décroissant de l'importance SHAP (en valeur absolue)
shap_summary = shap_summary.sort_values(by='Mean SHAP', ascending=False)
print(shap_summary)

Les `valeurs SHAP` permettent de comprendre l'impact de chaque caractéristique sur les prédictions du modèle. Les résultats obtenus pour `Academic_Pressure`, `Suicidal_Thoughts` et `Financial_Stress` indiquent qu'**une pression académique, des pensées suicidaires et du stress financier augmentent la probabilité de dépression (et inversement si les facteurs sont à un niveau bas**). Ces facteurs sont donc des indicateurs clés de l'état mental d'un étudiant. `Dietary_Habits_Unhealthy` a également un effet positif, mais moins marqué, suggérant que de mauvaises habitudes alimentaires peuvent aussi être liées à une détresse mentale. **L'influence de `Age` sur la prédiction est en quelque sorte "bipolaire"** : plus l'âge est élevé, plus l'impact sur la prédiction devient négatif, c'est-à-dire qu'il tend à réduire le risque que l'étudiant soit dépressif, tandis que des âges plus jeunes sont associés à des détresses mentales. **Ces résultats confirment ceux obtenus avec la `feature importance`.**

**Interprétation locale** : `Individual Conditional Expectation` ou ICE (comment la prédiction d'une instance est impactée si on fait varier la valeur d'une feature)

In [ ]:
# ICE
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# Après l'entraînement du modèle avec RandomizedSearchCV
best_model = random_search.best_estimator_

# Sélectionner une variable 
features_to_plot = ['Dietary_Habits_Unhealthy'] # exemple

# Tracer les courbes ICE pour la caractéristique
fig, ax = plt.subplots(figsize=(10, 8))
display = PartialDependenceDisplay.from_estimator(
    best_model, X_train, features=features_to_plot, kind="individual", grid_resolution=50, ax=ax
)

# Ajouter un titre et ajuster les labels
plt.title('ICE (Individual Conditional Expectation) for Dietary_Habits_Unhealthy')
plt.xlabel('Dietary_Habits_Unhealthy')
plt.ylabel('Probability of depression (model prediction)')
plt.show()

Dans la plupart des cas, les **`courbes ICE` ont une trajectoire similaire**, ce qui indique que **l'effet observé est relativement homogène parmi les individus (des habitudes alimentaires malsaines tendent à augmenter la probabilité de dépression chez l'ensemble des étudiants**).

##### Interprétation des résultats du modèle de régression logistique (meilleur modèle)

In [ ]:
# Résumé du modèle de régression logistique
print(result.summary())

In [ ]:
# Indicateurs de performance du modèle 

# Matrice de confusion
print("Confusion Matrix:\n", conf_matrix)
# Précision
print("Accuracy:", round(accuracy,4))
# Taux de sensibilité
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
print("Sensitivity:", round(sensitivity, 4))
# Taux de spécificité 
specificity = tn / (tn + fp)
print("Specificity:", round(specificity, 4))
# Score F1
print("F1 Score:", round(f1, 4))
# Pseudo R2
print("Pseudo R2:", round(1 - (result.llf / result.llnull),4)) 
# AUC-ROC
print("AUC-ROC:", round(roc_auc, 4))

Le modèle de `régression logistique` permet d'**analyser les facteurs influençant la probabilité de souffrir de dépression**. Le `pseudo R²` de 0.4857 indique que **près de 49 % de la variabilité de la dépression est expliquée par le modèle**, ce qui est relativement élevé pour cette méthode. Parmi les variables explicatives, des facteurs comme la pression académique (`Academic_Pressure`), les pensées suicidaires (`Suicidal_Thoughts`), le stress financier (`Financial_Stress`) et certains habitudes alimentaires (`Dietary_Habits_Unhealthy`) ont des **effets significatifs**, avec des coefficients positifs, indiquant qu'ils augmentent la probabilité de dépression. En revanche, des variables comme l'âge (`Age`) montrent des effets négatifs.

Les résultats des indicateurs de performance montrent que le **modèle a classifié correctement 84.69 % des cas** (`précision`). Nous pouvons également lire que le modèle détecte correctement la dépression dans 89.57 % des cas où elle est présente (`sensibilité`). Par contre, il distingue un peu moins bien les individus non dépressifs (`spécificité` est de 77.9 %). Le `score F1` de 0.8719 reflète un **équilibre relativement bon entre la précision et la sensibilité**. Enfin, l'`AUC-ROC` de 0.9228 montre une très bonne capacité de discrimination entre les individus souffrant de dépression et ceux ne souffrant pas. Les effets marginaux, calculés ci-après, permettront une interprétation plus précise des relations entre les variables et la probabilité de dépression.

In [ ]:
# Effets des variables explicatives

# Calcul des effets marginaux
marginal_effects = result.get_margeff()

# Résumé des effets marginaux avec p-values
print(marginal_effects.summary())

Les `effets marginaux` sont une méthode d'analyse utilisée pour interpréter les résultats d'un modèle de régression logistique en fournissant l'**impact d'une variation d'une unité d'une variable explicative sur la probabilité d'un événement (ici, la `dépression`)**. Voici l'interprétation des principaux résultats obtenus :

**Des `pensées suicidaires` (0.2757, p < 0.0001), des `habitudes alimentaires malsaines` (0.1154, p < 0.0001) et la `pression académique` (0.0917, p < 0.0001) augmentent fortement la probabilité de `dépression`** (effet marginal positif important et significatif). Concrètement cela signifie par exemple qu'une augmentation d'une unité sur l'échelle de la pression scolaire ressentie (par rapport à la valeur moyenne de 3 environ) augmente de 0.09 la probabilité que l'étudiant souffre de dépression. 

On recense **peu de variables tendant à réduire la probabilité de `dépression` chez les étudiants**. Dans ce cas de figure on trouve notamment la `satisfaction dans les études` (-0.0257, p < 0.0001).
En conclusion, **certains facteurs jouent un rôle prépondérant pour expliquer la `dépression` chez les étudiants indiens interrogés**, comme les pensées suicidaires, les habitudes alimentaires, la pression académique. Le **sens des relations significatives identifiées est presque toujours cohérent**. Plusieurs variables semblent être peu pertinentes pour expliquer le phénomène étudié, telles que la taille de la ville et le sexe. **Ces résultats confirment ceux obtenus avec la méthode `XGBoost`.**

### CONCLUSION

Dans ce projet, nous avons cherché à **comprendre et prédire la présence de dépression chez les étudiants en Inde** en exploitant un dataset de **27 901 observations** intégrant des informations **démographiques, académiques et comportementales**. Pour identifier les **facteurs les plus influents**, nous avons mobilisé **deux approches complémentaires** : **XGBoost**, reconnu pour sa capacité à capturer des relations complexes, et **la régression logistique**, privilégiée pour son interprétabilité et son adéquation aux problèmes de classification binaire.  

Après un **prétraitement minutieux des données** (gestion des valeurs atypiques, transtypage…) et une **analyse statistique approfondie**, nous avons entraîné et ajusté les modèles en veillant notamment au **traitement de la multicolinéarité**.  

L’évaluation des performances a montré que **XGBoost offrait une meilleure capacité prédictive**. Cependant, l’analyse des relations entre les **facteurs influents** et l’état de dépression – notamment via le **beeswarm plot** – a révélé une forte linéarité des effets. Dans ce contexte, **la régression logistique s’impose comme l’outil le plus pertinent**, car elle permet d’identifier des relations similaires tout en offrant une **interprétation claire et exploitable**.  

Dans un domaine aussi sensible que **la santé mentale**, la **transparence** des modèles est primordiale pour garantir une **compréhension accessible** par les professionnels et faciliter l’**élaboration de politiques publiques adaptées**. Contrairement à XGBoost, qui agit comme une **"boîte noire"**, la régression logistique permet de **quantifier directement l’impact des facteurs**, renforçant ainsi la **fiabilité et l’applicabilité des conclusions**.  

Nos résultats mettent en avant **quatre facteurs déterminants** dans la prédiction de la dépression : **la pression académique**, reflet des attentes élevées pesant sur les étudiants ; **les pensées suicidaires**, indicateur direct d’une détresse psychologique sévère ; **le stress financier**, qui accentue l’anxiété et le mal-être ; et enfin **une alimentation déséquilibrée**, soulignant un lien potentiel entre l’hygiène de vie et la santé mentale.  

Ces résultats confirment les tendances mises en avant dans la littérature et rappellent l’importance de prendre en compte ces facteurs dans les **stratégies de prévention et d’accompagnement des étudiants en difficulté**.

### DISCUSSION

Une limite importante à la généralisation de nos résultats est que l'ensemble de données se concentre sur **une seule année** (par ailleurs non renseignée). Cela restreint l’interprétation des résultats à une **photographie statique** de la situation des étudiants et empêche toute **analyse longitudinale**. En conséquence, il est impossible d’examiner la dynamique des facteurs influençant la dépression ou d’évaluer l’impact de chocs exogènes comme la pandémie de COVID-19. 

De plus, nous pourrions chercher à **accroître sa portée en étendant l'analyse à d’autres pays** afin d’évaluer la robustesse des résultats dans des contextes variés. Il serait également pertinent d'**intégrer des variables concernant l'entourage de l'étudiant**, comme le soutien social, qui peut jouer un rôle important dans la santé mentale.

Les limites soulevées sont finalement des **opportunités d'amélioration**, permettant d’accroître la compréhension des facteurs de risque pour la santé mentale des étudiants et d'affiner les politiques publiques mises en place.